In [ ]:
# set HF_HOME or HF_HUB_CACHE
import os
os.environ["HF_HOME"] = "/home/234533@hertie-school.lan/workspace/cache"
os.environ["HF_HUB_CACHE"] = "/home/234533@hertie-school.lan/workspace/cache"
os.environ['TRANSFORMERS_CACHE'] = "/home/234533@hertie-school.lan/workspace/cache"

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from datasets import Dataset
from sklearn.metrics import accuracy_score, classification_report

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load data
df = pd.read_csv('informal_reviewed.csv')
texts = df['content'].tolist()
df['lable'] = df['lable'].astype(int)
labels = df['lable'].tolist()


# Create dataset function - simplified for classification
def create_dataset_entries(texts, labels=None):
    if labels:
        return [{"text": text, "label": label} for text, label in zip(texts, labels)]
    else:
        return [{"text": text} for text in texts]

# Create training dataset
data_entries = create_dataset_entries(texts, labels)
train_val_split = int(len(data_entries) * 0.9)
train_entries, val_entries = data_entries[:train_val_split], data_entries[train_val_split:]

train_dataset = Dataset.from_list(train_entries)
val_dataset = Dataset.from_list(val_entries)

print(f"Training on {len(train_dataset)} examples, validating on {len(val_dataset)} examples")

# Load model and tokenizer
model_name = "microsoft/Phi-4-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir='/home/234533@hertie-school.lan/workspace/cache')
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Process datasets with tokenizer
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Load model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,  # Binary classification
    torch_dtype=torch.bfloat16,
    device_map="auto",
    cache_dir="/home/234533@hertie-school.lan/workspace/cache",
    problem_type="single_label_classification"
)
# Configure LoRA fine-tuning
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Change to sequence classification
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[
        "qkv_proj",     # Combined query-key-value projection
        "o_proj",       # Output projection
        "gate_up_proj", # MLP gate and up projections
        "down_proj" ]
)

# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Set up data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Define metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Training arguments
training_args = TrainingArguments(
    output_dir="./phi-dogwhistle-detector",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    num_train_epochs=3,
    warmup_ratio=0.05,
    fp16=False,
    bf16=True,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none",
    push_to_hub=False,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

# Save the model
trainer.save_model("./phi-dogwhistle-detector-final")

# Prediction function
def predict_dogwhistle(text, model, tokenizer, device):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
    
    return predictions.item()

# Load the fine-tuned model for inference
tokenizer_inference = AutoTokenizer.from_pretrained("./phi-dogwhistle-detector-final")
model_inference = AutoModelForSequenceClassification.from_pretrained(
    "./phi-dogwhistle-detector-final", 
    device_map="auto"
)

# Test the model on some examples
test_texts = texts[:5]  # Just using a few examples for testing
for i, text in enumerate(test_texts):
    prediction = predict_dogwhistle(text, model_inference, tokenizer_inference, device)
    print(f"Text: {text}")
    print(f"Prediction: {'Dogwhistle' if prediction == 1 else 'Not a dogwhistle'}")
    print(f"Actual label: {'Dogwhistle' if labels[i] == 1 else 'Not a dogwhistle'}")
    print("-" * 50)

# Evaluate on the entire dataset
all_predictions = []
for text in tqdm(texts, desc="Evaluating"):
    prediction = predict_dogwhistle(text, model_inference, tokenizer_inference, device)
    all_predictions.append(prediction)

# Calculate accuracy
accuracy = accuracy_score(labels, all_predictions)
print(f"Overall accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(labels, all_predictions))